In [46]:
# Import all the relevant libraries

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

### First step is to obtain the dataset for New York City where the study is based on

In [48]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

### Next we download the data using the json library since it is a json file

In [51]:
with open('newyork_data.json') as json_data:
    nyc_data = json.load(json_data)

In [52]:
nyc_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

### As it is easier to clean and process data in a dataframe, the downloaded data is converted from json type to a pandas dataframe. As all the relevant data is in the features key, we use that and assign it to the neighborhoods_data variable

In [55]:
neighborhoods_data = nyc_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [56]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [57]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### We can see that the city has 5 boroughs. We then print the name of these 5 boroughs

In [60]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [58]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Using the coordinates of NYC, we visualize the various neighborhoods of NYC in a folium map.

In [59]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### As we can see from the above map, Queens and Staten Island and disconnected from the NYC centre and as per the wishes of the client, we only consider locations which are directly connected (as say nearby the centre) of NYC. Of course centre is an arbitrary term used here and not clearly defined but more from visualization. Exact centering purposes are out of the scope of this study.

In [93]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

In [62]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


#### For visualization purposes, we show the different neighborhoods in one of the boroughs (in this case-Manhattan)

In [63]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### The foursquare API is now used to explore the various neighborhoods of NYC

In [64]:
CLIENT_ID = 'U22Z53NJJ1SUU2XXIRYVRKSBFO3E3RA20TU1RZKG5DISHSDQ' # your Foursquare ID
CLIENT_SECRET = 'PFMWLP2202SILPSEWF3XJ2V4JYDIPV3GT2OTNLHT02OUCUN1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U22Z53NJJ1SUU2XXIRYVRKSBFO3E3RA20TU1RZKG5DISHSDQ
CLIENT_SECRET:PFMWLP2202SILPSEWF3XJ2V4JYDIPV3GT2OTNLHT02OUCUN1


### We limit our search to 25 venues and the radius to 750 m as per the wishes of the client

In [66]:
LIMIT = 25
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### We start our analysis from the borough of Bronx

In [67]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [69]:
bronx_venues.shape

(1226, 7)

In [70]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,25,25,25,25,25,25
Baychester,25,25,25,25,25,25
Bedford Park,25,25,25,25,25,25
Belmont,25,25,25,25,25,25
Bronxdale,25,25,25,25,25,25
Castle Hill,14,14,14,14,14,14
City Island,25,25,25,25,25,25
Claremont Village,25,25,25,25,25,25
Clason Point,8,8,8,8,8,8


In [72]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 160 uniques categories.


### We analyze each individual neighborhood of Bronx

In [74]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Exhibit,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Latin American Restaurant,Laundromat,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mattress Store,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Music Venue,Nightclub,Paella Restaurant,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
bronx_onehot.shape

(1226, 161)

In [78]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Exhibit,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Latin American Restaurant,Laundromat,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mattress Store,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Music Venue,Nightclub,Paella Restaurant,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Allerton,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.04,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.040000,0.040000,0.00,0.04,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.040000,0.000000,0.000000,0.00,0.00,0.00,0.120000,0.00,0.00,0.00,0.000000,0.040000,0.00,0.040000,0.00,0.000000,0.00,0.040000,0.00,0.00,0.00,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.04,0.00,0.00,0.000000,0.04,0.00,0.04,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.080000,0.000000,0.120000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.080000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Baychester,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.04,0.040000,0.040000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.040000,0.00,0.000000,0.04,0.00,0.00,0.00,0.000000,0.08,0.000000,0.000000,0.080000,0.00,0.00,0.00,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.04,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.040000,0.000000,0.000000,0.04,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.04,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.04,

### Listing the top 5 venues in every neighborhood of Bronx

In [79]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
            venue  freq
0      Donut Shop  0.12
1     Pizza Place  0.12
2        Pharmacy  0.08
3  Sandwich Place  0.08
4             Bar  0.04


----Baychester----
                venue  freq
0      Discount Store  0.08
1    Department Store  0.08
2  Spanish Restaurant  0.08
3       Shopping Mall  0.04
4              Bakery  0.04


----Bedford Park----
                venue  freq
0               Diner  0.16
1  Mexican Restaurant  0.08
2         Pizza Place  0.08
3         Supermarket  0.08
4              Garden  0.08


----Belmont----
                venue  freq
0  Italian Restaurant  0.20
1       Deli / Bodega  0.16
2         Pizza Place  0.16
3              Bakery  0.12
4   Food & Drink Shop  0.08


----Bronxdale----
                venue  freq
0         Pizza Place  0.16
1  Italian Restaurant  0.12
2                Bank  0.04
3  Spanish Restaurant  0.04
4  Frozen Yogurt Shop  0.04


----Castle Hill----
                  venue  freq
0        Baseball Field  0.14
1  

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Donut Shop,Pizza Place,Pharmacy,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Food,Martial Arts Dojo,Bar,Mexican Restaurant
1,Baychester,Discount Store,Spanish Restaurant,Department Store,Pet Store,Bank,Donut Shop,Mexican Restaurant,Shopping Mall,Steakhouse,Shoe Store
2,Bedford Park,Diner,Pizza Place,Garden,Mexican Restaurant,Supermarket,Bar,Coffee Shop,Sandwich Place,Deli / Bodega,Park
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Food & Drink Shop,Fish Market,Grocery Store,Market,Mexican Restaurant
4,Bronxdale,Pizza Place,Italian Restaurant,Pool Hall,Sushi Restaurant,Frozen Yogurt Shop,Juice Bar,Food Truck,Mexican Restaurant,Middle Eastern Restaurant,Park


### The k-means clustering analysis is performed to cluster the neighborhoods into different categories.

In [84]:
# set number of clusters
kclusters = 3

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 0, 0, 1, 0, 1, 1, 2], dtype=int32)

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged =bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Supermarket,Gas Station,Donut Shop,Caribbean Restaurant,Food,Pharmacy,Ice Cream Shop,Sandwich Place,Discount Store,Chinese Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2,Pizza Place,Fast Food Restaurant,Mattress Store,Pharmacy,Caribbean Restaurant,Sandwich Place,Seafood Restaurant,Department Store,Shopping Mall,Mobile Phone Shop
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Fast Food Restaurant,Diner,Historic Site,Breakfast Spot,Burger Joint,Supplement Shop,Shopping Mall,Seafood Restaurant,Bowling Alley
3,Bronx,Fieldston,40.895437,-73.905643,1,Bus Station,Plaza,Athletics & Sports,Playground,Coffee Shop,Pizza Place,Park,Home Service,Gourmet Shop,Dessert Shop
4,Bronx,Riverdale,40.890834,-73.912585,1,Pizza Place,Bar,Japanese Restaurant,Health & Beauty Service,Plaza,Coffee Shop,Park,Deli / Bodega,Restaurant,Sandwich Place


### THe clusters are visualized in a map.

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [87]:
bronx_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Supermarket,Gas Station,Donut Shop,Caribbean Restaurant,Food,Pharmacy,Ice Cream Shop,Sandwich Place,Discount Store,Chinese Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2,Pizza Place,Fast Food Restaurant,Mattress Store,Pharmacy,Caribbean Restaurant,Sandwich Place,Seafood Restaurant,Department Store,Shopping Mall,Mobile Phone Shop
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Fast Food Restaurant,Diner,Historic Site,Breakfast Spot,Burger Joint,Supplement Shop,Shopping Mall,Seafood Restaurant,Bowling Alley
3,Bronx,Fieldston,40.895437,-73.905643,1,Bus Station,Plaza,Athletics & Sports,Playground,Coffee Shop,Pizza Place,Park,Home Service,Gourmet Shop,Dessert Shop
4,Bronx,Riverdale,40.890834,-73.912585,1,Pizza Place,Bar,Japanese Restaurant,Health & Beauty Service,Plaza,Coffee Shop,Park,Deli / Bodega,Restaurant,Sandwich Place
5,Bronx,Kingsbridge,40.881687,-73.902818,1,Mexican Restaurant,Coffee Shop,Bar,Pizza Place,Caribbean Restaurant,Spanish Restaurant,Diner,Sandwich Place,Bakery,Beer Bar
6,Bronx,Woodlawn,40.898273,-73.867315,0,Pub,Bar,Deli / Bodega,Pizza Place,Italian Restaurant,Bakery,Rental Car Location,Diner,Shipping Store,Discount Store
7,Bronx,Norwood,40.877224,-73.879391,1,Park,Pizza Place,Caribbean Restaurant,Sandwich Place,Bank,Gym,Supplement Shop,Coffee Shop,Gym / Fitness Center,Restaurant
8,Bronx,Williamsbridge,40.881039,-73.857446,1,Pizza Place,Caribbean Restaurant,Gym,Pharmacy,Spa,Seafood Restaurant,Bar,Nightclub,Soup Place,Restaurant
9,Bronx,Baychester,40.866858,-73.835798,1,Discount Store,Spanish Restaurant,Department Store,Pet Store,Bank,Donut Shop,Mexican Restaurant,Shopping Mall,Steakhouse,Shoe Store


In [89]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Pub,Bar,Deli / Bodega,Pizza Place,Italian Restaurant,Bakery,Rental Car Location,Diner,Shipping Store,Discount Store
10,Pelham Parkway,Pizza Place,Chinese Restaurant,Sandwich Place,Italian Restaurant,Bank,Bakery,Pharmacy,Performing Arts Venue,Deli / Bodega,Donut Shop
11,City Island,American Restaurant,Seafood Restaurant,Bar,Italian Restaurant,Harbor / Marina,Music Venue,Smoke Shop,Spanish Restaurant,Boat or Ferry,French Restaurant
27,Throgs Neck,Italian Restaurant,Pizza Place,Deli / Bodega,Coffee Shop,Sports Bar,Beach,Tennis Court,Bar,Juice Bar,Pub
28,Country Club,Bakery,Sandwich Place,Dive Bar,Bank,Italian Restaurant,Gym / Fitness Center,Pharmacy,Park,Diner,Mobile Phone Shop
31,Van Nest,Pizza Place,Italian Restaurant,Supermarket,Hookah Bar,Cosmetics Shop,Bowling Alley,Bus Station,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop
32,Morris Park,Pizza Place,Coffee Shop,Deli / Bodega,Bar,Bakery,Fast Food Restaurant,Supermarket,Liquor Store,Sandwich Place,Restaurant
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Food & Drink Shop,Fish Market,Grocery Store,Market,Mexican Restaurant
35,North Riverdale,Pizza Place,Italian Restaurant,Sandwich Place,Coffee Shop,Bank,Pool,Bookstore,Sushi Restaurant,Restaurant,Bar
36,Pelham Bay,Italian Restaurant,Bakery,Convenience Store,Gym / Fitness Center,Thai Restaurant,Deli / Bodega,Diner,Mexican Restaurant,Donut Shop,Sports Bar


### The above cluster can be classified as one where the venues are primarily related to food

In [90]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Eastchester,Caribbean Restaurant,Fast Food Restaurant,Diner,Historic Site,Breakfast Spot,Burger Joint,Supplement Shop,Shopping Mall,Seafood Restaurant,Bowling Alley
3,Fieldston,Bus Station,Plaza,Athletics & Sports,Playground,Coffee Shop,Pizza Place,Park,Home Service,Gourmet Shop,Dessert Shop
4,Riverdale,Pizza Place,Bar,Japanese Restaurant,Health & Beauty Service,Plaza,Coffee Shop,Park,Deli / Bodega,Restaurant,Sandwich Place
5,Kingsbridge,Mexican Restaurant,Coffee Shop,Bar,Pizza Place,Caribbean Restaurant,Spanish Restaurant,Diner,Sandwich Place,Bakery,Beer Bar
7,Norwood,Park,Pizza Place,Caribbean Restaurant,Sandwich Place,Bank,Gym,Supplement Shop,Coffee Shop,Gym / Fitness Center,Restaurant
8,Williamsbridge,Pizza Place,Caribbean Restaurant,Gym,Pharmacy,Spa,Seafood Restaurant,Bar,Nightclub,Soup Place,Restaurant
9,Baychester,Discount Store,Spanish Restaurant,Department Store,Pet Store,Bank,Donut Shop,Mexican Restaurant,Shopping Mall,Steakhouse,Shoe Store
12,Bedford Park,Diner,Pizza Place,Garden,Mexican Restaurant,Supermarket,Bar,Coffee Shop,Sandwich Place,Deli / Bodega,Park
13,University Heights,Pizza Place,Grocery Store,Spanish Restaurant,Fast Food Restaurant,Bakery,Sandwich Place,Shoe Store,Mobile Phone Shop,Donut Shop,Supermarket
14,Morris Heights,Food Truck,Pizza Place,Grocery Store,Spanish Restaurant,Gym,Playground,Clothing Store,Restaurant,Chinese Restaurant,Music Venue


### The above cluster indicates that the venues here are more diverse and include a lot of fitness related stuff such as gyms, spas, fitness centres etc.

In [92]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Supermarket,Gas Station,Donut Shop,Caribbean Restaurant,Food,Pharmacy,Ice Cream Shop,Sandwich Place,Discount Store,Chinese Restaurant
1,Co-op City,Pizza Place,Fast Food Restaurant,Mattress Store,Pharmacy,Caribbean Restaurant,Sandwich Place,Seafood Restaurant,Department Store,Shopping Mall,Mobile Phone Shop
22,Longwood,Grocery Store,Donut Shop,Fast Food Restaurant,Sandwich Place,Discount Store,Latin American Restaurant,Coffee Shop,Theater,Pizza Place,Pharmacy
24,Morrisania,Discount Store,Donut Shop,Fast Food Restaurant,Pharmacy,Grocery Store,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Bowling Alley,Supermarket
25,Soundview,Fast Food Restaurant,Clothing Store,Supermarket,Liquor Store,Fried Chicken Joint,Latin American Restaurant,Kids Store,Donut Shop,Mobile Phone Shop,American Restaurant
40,Olinville,Pharmacy,Caribbean Restaurant,Donut Shop,Fast Food Restaurant,Sandwich Place,Bank,Mobile Phone Shop,Mexican Restaurant,Brewery,Breakfast Spot
41,Pelham Gardens,Donut Shop,Pharmacy,Bus Station,Locksmith,Pet Store,Dance Studio,Deli / Bodega,Chinese Restaurant,Sandwich Place,Café
42,Concourse,Donut Shop,Grocery Store,Ice Cream Shop,Discount Store,Bar,Spanish Restaurant,Mobile Phone Shop,Pizza Place,Fast Food Restaurant,Caribbean Restaurant
44,Edenwald,Supermarket,Caribbean Restaurant,Pharmacy,Fast Food Restaurant,Bakery,Pizza Place,Sandwich Place,Nightclub,Donut Shop,Bar
47,Mount Eden,Grocery Store,Pizza Place,Ice Cream Shop,Donut Shop,Bar,Pharmacy,Sandwich Place,Deli / Bodega,Nightclub,Music Venue


### The final cluster again indicates more venues related to food but different from the first cluster where Italian Cuisine seemed more popular. In this cluster fastfoods, donuts ice-creams seem more popular.

### The entire process is repeated again for the different boroughs such as Manhattan and Brooklyn. As the process is essentially the same, comments from here on are limited.

In [94]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [96]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0   Memorial Site  0.12
1            Park  0.12
2      Food Court  0.08
3     Coffee Shop  0.08
4  Sandwich Place  0.04


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.08
1  Gym / Fitness Center  0.08
2                   Gym  0.08
3             Bookstore  0.08
4    Italian Restaurant  0.08


----Central Harlem----
                  venue  freq
0     French Restaurant  0.08
1   American Restaurant  0.08
2             Jazz Club  0.04
3  Gym / Fitness Center  0.04
4             Bookstore  0.04


----Chelsea----
                venue  freq
0  Italian Restaurant  0.08
1      Ice Cream Shop  0.08
2             Theater  0.08
3   Indian Restaurant  0.04
4              Market  0.04


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.08
1                 Spa  0.08
2      Ice Cream Shop  0.08
3      Sandwich Place  0.08
4  English Restaurant  0.04


----Civic Center----
                    venu

In [97]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Memorial Site,Park,Food Court,Coffee Shop,Ice Cream Shop,Grocery Store,Shopping Mall,Cupcake Shop,Scenic Lookout,Sandwich Place
1,Carnegie Hill,Bookstore,Gym / Fitness Center,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Dance Studio,Concert Hall,Restaurant,Café
2,Central Harlem,French Restaurant,American Restaurant,Boutique,Chinese Restaurant,Beer Bar,Fried Chicken Joint,Bookstore,Food Truck,Café,Music Venue
3,Chelsea,Ice Cream Shop,Italian Restaurant,Theater,Chinese Restaurant,Bar,Nightclub,Taco Place,Seafood Restaurant,French Restaurant,Coffee Shop
4,Chinatown,Ice Cream Shop,Chinese Restaurant,Sandwich Place,Spa,Thai Restaurant,Pizza Place,Cocktail Bar,Noodle House,New American Restaurant,Museum


In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [103]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
            venue  freq
0     Pizza Place  0.16
1          Bakery  0.08
2     Coffee Shop  0.08
3    Dessert Shop  0.04
4  Shopping Plaza  0.04


----Bay Ridge----
                venue  freq
0                 Spa  0.12
1       Grocery Store  0.08
2  Italian Restaurant  0.08
3    Greek Restaurant  0.08
4         Pizza Place  0.08


----Bedford Stuyvesant----
         venue  freq
0    Wine Shop  0.12
1          Bar  0.12
2  Coffee Shop  0.08
3    Juice Bar  0.08
4  Pizza Place  0.08


----Bensonhurst----
                 venue  freq
0   Italian Restaurant  0.12
1           Bagel Shop  0.08
2  Japanese Restaurant  0.08
3      Supplement Shop  0.08
4   Chinese Restaurant  0.08


----Bergen Beach----
                        venue  freq
0             Harbor / Marina  0.22
1              Baseball Field  0.11
2                  Donut Shop  0.11
3           Other Repair Shop  0.11
4  Construction & Landscaping  0.11


----Boerum Hill----
                    venue  freq
0    

In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pizza Place,Bakery,Coffee Shop,Burger Joint,Park,Beach,Shopping Plaza,Sushi Restaurant,Surf Spot,Spa
1,Bay Ridge,Spa,Pizza Place,Greek Restaurant,Italian Restaurant,Grocery Store,Cosmetics Shop,Coffee Shop,Caucasian Restaurant,Sports Bar,Breakfast Spot
2,Bedford Stuyvesant,Wine Shop,Bar,Juice Bar,Pizza Place,Coffee Shop,Cocktail Bar,Caribbean Restaurant,Bakery,BBQ Joint,Park
3,Bensonhurst,Italian Restaurant,Bagel Shop,Ice Cream Shop,Chinese Restaurant,Japanese Restaurant,Supplement Shop,Bakery,Supermarket,Cosmetics Shop,Coffee Shop
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Other Repair Shop,Playground,Construction & Landscaping,Gym,Baseball Field,Donut Shop,Women's Store,Farmers Market


In [105]:
# set number of clusters
kclusters = 3

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [110]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Pizza Place,Greek Restaurant,Italian Restaurant,Grocery Store,Cosmetics Shop,Coffee Shop,Caucasian Restaurant,Sports Bar,Breakfast Spot
1,Bensonhurst,Italian Restaurant,Bagel Shop,Ice Cream Shop,Chinese Restaurant,Japanese Restaurant,Supplement Shop,Bakery,Supermarket,Cosmetics Shop,Coffee Shop
2,Sunset Park,Pizza Place,Latin American Restaurant,Mexican Restaurant,Bank,Bakery,Record Shop,Deli / Bodega,Park,Coffee Shop,Café
3,Greenpoint,Bar,Cocktail Bar,Mexican Restaurant,Deli / Bodega,Bookstore,Boutique,Sushi Restaurant,Café,Coffee Shop,Restaurant
4,Gravesend,Bakery,Pizza Place,Pharmacy,Pool Hall,Lounge,Bar,Bagel Shop,Donut Shop,Chinese Restaurant,Turkish Restaurant
5,Brighton Beach,Beach,Russian Restaurant,Varenyky restaurant,Non-Profit,Sushi Restaurant,Theater,Mediterranean Restaurant,Bookstore,Other Great Outdoors,Supermarket
6,Sheepshead Bay,Turkish Restaurant,Sandwich Place,Sushi Restaurant,Restaurant,Italian Restaurant,Grocery Store,Gourmet Shop,Café,Seafood Restaurant,Chinese Restaurant
7,Manhattan Terrace,Pizza Place,Sushi Restaurant,Steakhouse,Bagel Shop,Russian Restaurant,Burrito Place,Middle Eastern Restaurant,Bakery,Donut Shop,Turkish Restaurant
9,Crown Heights,Café,Pizza Place,Museum,Yoga Studio,Bagel Shop,Comic Shop,Salon / Barbershop,Other Great Outdoors,Caribbean Restaurant,Candy Store
11,Kensington,Thai Restaurant,Mexican Restaurant,Café,Grocery Store,Veterinarian,Lingerie Store,Bagel Shop,Park,Flower Shop,Donut Shop


In [111]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Caribbean Restaurant,Coffee Shop,Juice Bar,Mexican Restaurant,Café,Pizza Place,Gym,Tapas Restaurant,Tibetan Restaurant,Burger Joint
10,East Flatbush,Caribbean Restaurant,Discount Store,Supermarket,Lounge,Deli / Bodega,Performing Arts Venue,Fruit & Vegetable Store,Liquor Store,Mexican Restaurant,Chinese Restaurant
14,Brownsville,Pizza Place,Discount Store,Supermarket,School,Mobile Phone Shop,Moving Target,Fish & Chips Shop,Spanish Restaurant,Caribbean Restaurant,Sandwich Place
25,Cypress Hills,Donut Shop,Fried Chicken Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Latin American Restaurant,Ice Cream Shop,South American Restaurant,Supermarket,Liquor Store
28,Canarsie,Caribbean Restaurant,Bank,Chinese Restaurant,Asian Restaurant,Food,Fast Food Restaurant,Grocery Store,Bakery,Bagel Shop,Gym
29,Flatlands,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Lounge,Caribbean Restaurant,Mobile Phone Shop,Park,Music Venue,Nightclub,Bar
43,Ocean Hill,Discount Store,Ice Cream Shop,Coffee Shop,Chinese Restaurant,Café,Supermarket,Bakery,Pizza Place,Restaurant,Donut Shop
44,City Line,Pizza Place,Bank,Donut Shop,Shoe Store,Fried Chicken Joint,Food Truck,Mobile Phone Shop,South American Restaurant,Fast Food Restaurant,Yoga Studio
48,Georgetown,Bank,American Restaurant,Women's Store,Chinese Restaurant,Frozen Yogurt Shop,Miscellaneous Shop,Discount Store,Burger Joint,Café,Coffee Shop
52,Ocean Parkway,Steakhouse,Discount Store,Café,Fast Food Restaurant,Donut Shop,Paper / Office Supplies Store,Turkish Restaurant,Supermarket,Falafel Restaurant,Lounge


In [112]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Caribbean Restaurant,Coffee Shop,Juice Bar,Mexican Restaurant,Café,Pizza Place,Gym,Tapas Restaurant,Tibetan Restaurant,Burger Joint
10,East Flatbush,Caribbean Restaurant,Discount Store,Supermarket,Lounge,Deli / Bodega,Performing Arts Venue,Fruit & Vegetable Store,Liquor Store,Mexican Restaurant,Chinese Restaurant
14,Brownsville,Pizza Place,Discount Store,Supermarket,School,Mobile Phone Shop,Moving Target,Fish & Chips Shop,Spanish Restaurant,Caribbean Restaurant,Sandwich Place
25,Cypress Hills,Donut Shop,Fried Chicken Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Latin American Restaurant,Ice Cream Shop,South American Restaurant,Supermarket,Liquor Store
28,Canarsie,Caribbean Restaurant,Bank,Chinese Restaurant,Asian Restaurant,Food,Fast Food Restaurant,Grocery Store,Bakery,Bagel Shop,Gym
29,Flatlands,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Lounge,Caribbean Restaurant,Mobile Phone Shop,Park,Music Venue,Nightclub,Bar
43,Ocean Hill,Discount Store,Ice Cream Shop,Coffee Shop,Chinese Restaurant,Café,Supermarket,Bakery,Pizza Place,Restaurant,Donut Shop
44,City Line,Pizza Place,Bank,Donut Shop,Shoe Store,Fried Chicken Joint,Food Truck,Mobile Phone Shop,South American Restaurant,Fast Food Restaurant,Yoga Studio
48,Georgetown,Bank,American Restaurant,Women's Store,Chinese Restaurant,Frozen Yogurt Shop,Miscellaneous Shop,Discount Store,Burger Joint,Café,Coffee Shop
52,Ocean Parkway,Steakhouse,Discount Store,Café,Fast Food Restaurant,Donut Shop,Paper / Office Supplies Store,Turkish Restaurant,Supermarket,Falafel Restaurant,Lounge
